In [9]:
from google.colab import drive
drive.mount('/content/drive')
data_root= '/content/drive/My Drive/chatbot/intents.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
data_file=open(data_root,'r').read()
intents=json.loads(data_file)

In [12]:
# Data Preparation

words = []
classes = []
data_X = []
data_Y = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(tokens)
        data_Y.append(intent['tag'])

    if intent['tag'] not in classes:
        classes.append(intent['tag'])

lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))

classes = sorted(set(classes))

print("Words:", words)
print("Classes:", classes)

training = []
out_empty = [0] * len(classes)

for idx, doc in enumerate(data_X):
    bow = []
    text = " ".join(doc)
    text = lemmatizer.lemmatize(text.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    output_row = list(out_empty)
    output_row[classes.index(data_Y[idx])] = 1

    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

print("Training X shape:", train_X.shape)
print("Training Y shape:", train_Y.shape)

# Neural Network

model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=200, verbose=1)

# Preprocessing the unit

def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    print("Predicted classes:", return_list)  # Debugging print
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I didn't get that."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    return result

# Interaction with Chatbot

print("Press 0 if you don't want to chat with our chatbot")
while True:
    message = input("")
    if message == "0":
        break

    intents_list = pred_class(message, words, classes)
    result = get_response(intents_list, intents)
    print("Response:", result)


Words: ["'s", '2', 'a', 'afternoon', 'age', 'am', 'are', 'ask', 'bad', 'born', 'boy', 'bye', 'can', 'do', 'doing', 'feel', 'feeling', 'fine', 'girl', 'good', 'goodbye', 'great', 'hello', 'help', 'here', 'hi', 'how', 'i', 'is', 'job', 'live', 'me', 'men', 'minute', 'morning', 'much', 'no', 'old', 'please', 'sad', 'sec', 'still', 'thank', 'thanks', 'there', 'today', 'too', 'up', 'very', 'wait', 'were', 'what', 'when', 'where', 'woman', 'work', 'you', 'your']
Classes: ['action', 'actions', 'age', 'bad', 'city', 'feeling', 'good', 'goodbye', 'hello', 'job', 'men', 'noanswer', 'still there', 'thanks', 'wait', 'women']
Training X shape: (41, 58)
Training Y shape: (41, 16)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               7552      
                                                                 
 dropout_2 (Dropout)         (None, 128